**Part-A Q2**

**Training the model and sweep on various hyperparameter configurations**

## Importing libraries and dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Activation,MaxPooling2D,Flatten,BatchNormalization,Dropout,InputLayer
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

## Function for creating CNN Model using parameters given

In [4]:
def create_cnn_model(filters, fil_size, dense_neuron, batch_norm, dropout, act, padd, lr):
    keras.backend.clear_session()
    model = Sequential()
    model.add(InputLayer(input_shape=(227,227,3))) #Adding Input layer with image size as(227,227) with RGB channel

    #Adding 5 convolution Layers in a loop
    for i in range(5):
        model.add(Conv2D(filters[i],fil_size[i],padding=padd))
        if batch_norm:
            model.add(BatchNormalization())   
        model.add(Activation(act))
        model.add(MaxPooling2D(pool_size=(2,2)))
    
    #Adding the Dense layers as per the parameters passed
    model.add(Flatten())
    model.add(Dense(dense_neuron))
    model.add(Dropout(dropout))
    model.add(Activation("relu"))

    #Taking Adam as optimizer for all and not as a hyperparamater.
    model.add(Dense(10,activation='softmax'))
    adam = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

    return model

## Definition Function for preparing DATA:-

In [5]:
#image size is taken as (227,227)

def dataProcess(BATCH_SIZE, data_aug=True):
    if(data_aug):
        train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=90, shear_range=0.2, zoom_range=0.2, validation_split=0.1, horizontal_flip=True)
    else:
        train_datagen = ImageDataGenerator( rescale=1./255, validation_split=0.1)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/DL_DATA_iNat/train',
        subset='training',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
        seed = 45)

    val_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/DL_DATA_iNat/train',
        subset = 'validation',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
        seed = 45)

    test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/DL_DATA_iNat/val',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
    return train_generator, val_generator, test_generator
    

## WandB install and login

In [ ]:
!pip install wandb


In [7]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Fetch data, Train and sweep :-

In [8]:
#--------Fetching dataset-------------------------------------------------------

#--Batch size set by the user
BATCH_SIZE = 32
train_gen, val_gen, test_gen = dataProcess(BATCH_SIZE, True)    #Data augmentation is kept true and not as hyperparameter.

Found 9018 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2009 images belonging to 10 classes.


In [ ]:
# Sweep configurations ---------------------------------------------------------

sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': { 'values': [8]
        },
        'filters': {
            'values': [[16,32,64,128,256], [32,64,128,256,256],
                       [64,32,16,8,4], [32,16,8,4,2],
                       [64,64,64,64,64], [32,32,32,32,32], [48,48,48,48,48]
                      ]
        },
        'filter_size': { 'values': [  [(7,7),(7,7),(7,7),(7,7),(7,7)],
                                      [(5,5),(5,5),(5,5),(5,5),(5,5)],
                                      [(3,3),(3,3),(3,3),(3,3),(3,3)],
                                      [(2,2),(2,2),(2,2),(2,2),(2,2)]  ]
        },
        'dense_neuron': { 'values': [256,512,1024]
        },
        'batch_norm': { 'values': [True,False]
        },
        'dropout': { 'values': [0.15,0.2]
        },
        'lr': { 'values': [1e-4,1e-5]
        }
    }
}

In [9]:
import wandb
from wandb.keras import WandbCallback

def train():
    config_defaults = {
        "filters": [32,64,128,256,256],
        "filter_size": [(3,3),(3,3),(3,3),(3,3),(3,3)],
        "dropout": 0.15,
        "batch_norm": True,
        "epochs": 10,
        "dense_neuron": 512,
        "lr": 0.0001
    }

    wandb.init(config=config_defaults,name="CS6910-A2-P_A")
    config = wandb.config
    wandb.run.name = "flt_{}_fltsz_{}_do_{}_bn_{}_dsz_{}_lr_{}".format(config.filters,\
                                                      config.filter_size[0],\
                                                      config.dropout,\
                                                      config.batch_norm,\
                                                      config.dense_neuron,\
                                                      config.lr)
    wandb.run.save()
    model = create_cnn_model(config.filters, config.filter_size, config.dense_neuron, config.batch_norm, config.dropout, 'relu', "same", config.lr)
    #Activation is done by ReLu only and not as a hyperparamater, but can be set by the user a parameter
    #padding is also set to "same" by default.

    trained_arti = model.fit(
                              train_gen,
                              steps_per_epoch = train_gen.samples // BATCH_SIZE,
                              validation_data = val_gen, 
                              validation_steps = val_gen.samples // BATCH_SIZE,
                              epochs = config.epochs, 
                              callbacks=[WandbCallback(monitor='val_accuracy',mode='max')]
                            )
    #model.save('/content/drive/MyDrive/DL_DATA_iNat/best_model.h5)
    wandb.finish()
    return model, trained_arti

In [ ]:
sweep_id = wandb.sweep(sweep_config,entity="pandp",project = 'cs6910-As-2')

Create sweep with ID: et1l06k6
Sweep URL: https://wandb.ai/pandp/cs6910-As-2/sweeps/et1l06k6


In [10]:
sweep_id = 'et1l06k6'

Create sweep with ID: et1l06k6
Sweep URL: https://wandb.ai/pandp/cs6910-As-2/sweeps/et1l06k6

In [ ]:
wandb.agent(sweep_id, train, count=40)

wandb: Agent Starting Run: zux3nesi with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 512
wandb: 	dropout: 0.2
wandb: 	epochs: 8
wandb: 	filter_size: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	filters: [64, 32, 16, 8, 4]
wandb: 	lr: 0.0001
wandb: Currently logged in as: pranab045 (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Epoch 1/8
281/281 [==============================] - 2645s 9s/step - loss: 2.2556 - accuracy: 0.1710 - val_loss: 2.5133 - val_accuracy: 0.1018 - _timestamp: 1648645910.0000 - _runtime: 2662.0000
Epoch 2/8
281/281 [==============================] - 278s 989ms/step - loss: 2.1342 - accuracy: 0.2221 - val_loss: 2.1438 - val_accuracy: 0.2188 - _timestamp: 1648646206.0000 - _runtime: 2958.0000
Epoch 3/8
281/281 [==============================] - 268s 954ms/step - loss: 2.0881 - accuracy: 0.2425 - val_loss: 2.0696 - val_accuracy: 0.2520 - _timestamp: 1648646474.0000 - _runtime: 3226.0000
Epoch 4/8
281/281 [==============================] - 275s 977ms/step - loss: 2.0442 - accuracy: 0.2674 - val_loss: 2.0737 - val_accuracy: 0.2460 - _timestamp: 1648646803.0000 - _runtime: 3555.0000
Epoch 5/8
281/281 [==============================] - 268s 952ms/step - loss: 2.0161 - accuracy: 0.2729 - val_loss: 2.0066 - val_accuracy: 0.2812 - _timestamp: 1648647070.0000 - _runtime: 3822.0000
Epoch 6/8
281/281

accuracy,▁▄▅▆▇▇██
epoch,▁▂▃▄▅▆▇█
loss,█▅▄▃▂▂▁▁
val_accuracy,▁▅▆▆▇█▇█
val_loss,█▃▂▂▂▁▁▁
accuracy,0.2978
best_epoch,7
best_val_accuracy,0.30141
epoch,7
loss,1.97022
val_accuracy,0.30141


wandb: Agent Starting Run: d7tp640m with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 1024
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	filters: [64, 32, 16, 8, 4]
wandb: 	lr: 1e-05


Epoch 1/8
281/281 [==============================] - 280s 993ms/step - loss: 2.3649 - accuracy: 0.1118 - val_loss: 2.3161 - val_accuracy: 0.0938 - _timestamp: 1648648850.0000 - _runtime: 289.0000
Epoch 2/8
281/281 [==============================] - 282s 1s/step - loss: 2.2943 - accuracy: 0.1460 - val_loss: 2.2521 - val_accuracy: 0.1462 - _timestamp: 1648649132.0000 - _runtime: 571.0000
Epoch 3/8
281/281 [==============================] - 275s 977ms/step - loss: 2.2722 - accuracy: 0.1564 - val_loss: 2.2324 - val_accuracy: 0.1845 - _timestamp: 1648649446.0000 - _runtime: 885.0000
Epoch 4/8
281/281 [==============================] - 270s 960ms/step - loss: 2.2384 - accuracy: 0.1717 - val_loss: 2.2038 - val_accuracy: 0.1835 - _timestamp: 1648649716.0000 - _runtime: 1155.0000
Epoch 5/8
281/281 [==============================] - 266s 948ms/step - loss: 2.2084 - accuracy: 0.1933 - val_loss: 2.1927 - val_accuracy: 0.1865 - _timestamp: 1648649982.0000 - _runtime: 1421.0000
Epoch 6/8
247/281 [==

In [ ]:
wandb.agent(sweep_id,train,entity="pandp",project = 'cs6910-As-2', count=40)

wandb: Agent Starting Run: 9f4kx1fm with config:
wandb: 	batch_norm: False
wandb: 	dense_neuron: 256
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	filters: [64, 32, 16, 8, 4]
wandb: 	lr: 0.0001
wandb: Currently logged in as: pranab045 (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Epoch 1/8
 66/281 [======>.......................] - ETA: 30:15 - loss: 2.3034 - accuracy: 0.1018


Run 9f4kx1fm errored: UnknownError()
wandb: ERROR Run 9f4kx1fm errored: UnknownError()
wandb: Agent Starting Run: jbh5rbn1 with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 1024
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2]]
wandb: 	filters: [32, 16, 8, 4, 2]
wandb: 	lr: 0.0001


Epoch 1/8
281/281 [==============================] - 1867s 7s/step - loss: 2.2851 - accuracy: 0.1449 - val_loss: 2.3194 - val_accuracy: 0.1210 - _timestamp: 1648653208.0000 - _runtime: 1874.0000
Epoch 2/8
281/281 [==============================] - 756s 3s/step - loss: 2.1888 - accuracy: 0.1950 - val_loss: 2.2114 - val_accuracy: 0.1633 - _timestamp: 1648653980.0000 - _runtime: 2646.0000
Epoch 3/8
281/281 [==============================] - 749s 3s/step - loss: 2.1457 - accuracy: 0.2191 - val_loss: 2.1314 - val_accuracy: 0.2228 - _timestamp: 1648654775.0000 - _runtime: 3441.0000
Epoch 4/8
281/281 [==============================] - 756s 3s/step - loss: 2.1170 - accuracy: 0.2266 - val_loss: 2.1002 - val_accuracy: 0.2349 - _timestamp: 1648655583.0000 - _runtime: 4249.0000
Epoch 5/8
281/281 [==============================] - 768s 3s/step - loss: 2.1055 - accuracy: 0.2289 - val_loss: 2.0888 - val_accuracy: 0.2369 - _timestamp: 1648656398.0000 - _runtime: 5064.0000
Epoch 6/8
281/281 [==========

...

wandb: Agent Starting Run: g6dxxwwy with config:
wandb: 	batch_norm: False
wandb: 	dense_neuron: 1024
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	filters: [64, 32, 16, 8, 4]
wandb: 	lr: 0.0001


Epoch 1/8
281/281 [==============================] - 1120s 4s/step - loss: 2.2654 - accuracy: 0.1400 - val_loss: 2.2093 - val_accuracy: 0.1855 - _timestamp: 1648675235.0000 - _runtime: 1133.0000
Epoch 2/8
281/281 [==============================] - 1135s 4s/step - loss: 2.1847 - accuracy: 0.1971 - val_loss: 2.1628 - val_accuracy: 0.1976 - _timestamp: 1648676412.0000 - _runtime: 2310.0000
Epoch 3/8
281/281 [==============================] - 1138s 4s/step - loss: 2.1205 - accuracy: 0.2297 - val_loss: 2.0921 - val_accuracy: 0.2520 - _timestamp: 1648677577.0000 - _runtime: 3475.0000
Epoch 4/8
281/281 [==============================] - 1141s 4s/step - loss: 2.0691 - accuracy: 0.2647 - val_loss: 2.0695 - val_accuracy: 0.2651 - _timestamp: 1648678718.0000 - _runtime: 4616.0000
Epoch 5/8
281/281 [==============================] - 1146s 4s/step - loss: 2.0423 - accuracy: 0.2708 - val_loss: 2.0566 - val_accuracy: 0.2702 - _timestamp: 1648679864.0000 - _runtime: 5762.0000
Epoch 6/8
281/281 [======

accuracy,▁▄▅▇▇▇██
epoch,▁▂▃▄▅▆▇█
loss,█▆▄▃▂▂▁▁
val_accuracy,▁▂▅▆▆▇█▇
val_loss,█▆▄▃▃▂▁▁
accuracy,0.28778
best_epoch,6
best_val_accuracy,0.29435
epoch,7
loss,1.9829
val_accuracy,0.2752


wandb: Agent Starting Run: c91k7mkf with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 8
wandb: 	filter_size: [[7, 7], [7, 7], [7, 7], [7, 7], [7, 7]]
wandb: 	filters: [32, 64, 128, 256, 256]
wandb: 	lr: 0.0001


Epoch 1/8
281/281 [==============================] - 6525s 23s/step - loss: 2.1982 - accuracy: 0.2201 - val_loss: 2.5431 - val_accuracy: 0.0988 - _timestamp: 1648689888.0000 - _runtime: 6537.0000
Epoch 2/8
162/281 [================>.............] - ETA: 44:46 - loss: 2.0714 - accuracy: 0.2641

In [ ]:
wandb.agent(sweep_id,train,entity="pandp",project = 'cs6910-As-2', count=40)

wandb: Agent Starting Run: 68su0ypn with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 1024
wandb: 	dropout: 0.2
wandb: 	epochs: 8
wandb: 	filter_size: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	filters: [32, 64, 128, 256, 256]
wandb: 	lr: 0.0001
wandb: Currently logged in as: pranab045 (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Epoch 1/8
281/281 [==============================] - 2596s 9s/step - loss: 2.2532 - accuracy: 0.2381 - val_loss: 2.3279 - val_accuracy: 0.1240 - _timestamp: 1648745077.0000 - _runtime: 2614.0000
Epoch 2/8
281/281 [==============================] - 286s 1s/step - loss: 2.0011 - accuracy: 0.2861 - val_loss: 1.9998 - val_accuracy: 0.2883 - _timestamp: 1648745370.0000 - _runtime: 2907.0000
Epoch 3/8
281/281 [==============================] - 279s 992ms/step - loss: 1.9450 - accuracy: 0.3095 - val_loss: 1.9716 - val_accuracy: 0.3075 - _timestamp: 1648745685.0000 - _runtime: 3222.0000
Epoch 4/8
281/281 [==============================] - 280s 995ms/step - loss: 1.9027 - accuracy: 0.3209 - val_loss: 1.8648 - val_accuracy: 0.3498 - _timestamp: 1648745964.0000 - _runtime: 3501.0000
Epoch 5/8
281/281 [==============================] - 281s 1s/step - loss: 1.8718 - accuracy: 0.3376 - val_loss: 2.0012 - val_accuracy: 0.3004 - _timestamp: 1648746246.0000 - _runtime: 3783.0000
Epoch 6/8
281/281 [====

accuracy,▁▄▅▆▇▇██
epoch,▁▂▃▄▅▆▇█
loss,█▄▃▃▂▂▁▁
val_accuracy,▁▆▆▇▆▇▇█
val_loss,█▃▃▂▃▁▁▁
accuracy,0.36178
best_epoch,7
best_val_accuracy,0.36996
epoch,7
loss,1.80237
val_accuracy,0.36996


wandb: Agent Starting Run: 4boh9ge6 with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 1024
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[7, 7], [7, 7], [7, 7], [7, 7], [7, 7]]
wandb: 	filters: [32, 16, 8, 4, 2]
wandb: 	lr: 1e-05


...

wandb: Agent Starting Run: mffuzbbs with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 512
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[5, 5], [5, 5], [5, 5], [5, 5], [5, 5]]
wandb: 	filters: [32, 16, 8, 4, 2]
wandb: 	lr: 1e-05


Epoch 1/8
281/281 [==============================] - 268s 947ms/step - loss: 2.3896 - accuracy: 0.1056 - val_loss: 2.3136 - val_accuracy: 0.0988 - _timestamp: 1648768220.0000 - _runtime: 276.0000
Epoch 2/8
281/281 [==============================] - 265s 943ms/step - loss: 2.3492 - accuracy: 0.1091 - val_loss: 2.3110 - val_accuracy: 0.1179 - _timestamp: 1648768540.0000 - _runtime: 596.0000
Epoch 3/8
281/281 [==============================] - 270s 959ms/step - loss: 2.3387 - accuracy: 0.1197 - val_loss: 2.2988 - val_accuracy: 0.1240 - _timestamp: 1648768866.0000 - _runtime: 922.0000
Epoch 4/8
281/281 [==============================] - 269s 958ms/step - loss: 2.3237 - accuracy: 0.1242 - val_loss: 2.3005 - val_accuracy: 0.1411 - _timestamp: 1648769135.0000 - _runtime: 1191.0000
Epoch 5/8
281/281 [==============================] - 271s 964ms/step - loss: 2.3116 - accuracy: 0.1303 - val_loss: 2.2954 - val_accuracy: 0.1250 - _timestamp: 1648769459.0000 - _runtime: 1515.0000
Epoch 6/8
281/281 

accuracy,▁▂▃▄▅▆▆█
epoch,▁▂▃▄▅▆▇█
loss,█▅▅▄▃▂▂▁
val_accuracy,▁▃▄▆▄▅██
val_loss,██▆▆▅▄▂▁
accuracy,0.14834
best_epoch,6
best_val_accuracy,0.1623
epoch,7
loss,2.28377
val_accuracy,0.1623


wandb: Agent Starting Run: skeejhjq with config:
wandb: 	batch_norm: True
wandb: 	dense_neuron: 1024
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2]]
wandb: 	filters: [32, 64, 128, 256, 256]
wandb: 	lr: 1e-05


Epoch 1/8
281/281 [==============================] - 275s 976ms/step - loss: 2.3151 - accuracy: 0.2081 - val_loss: 2.4917 - val_accuracy: 0.1190 - _timestamp: 1648770617.0000 - _runtime: 282.0000
Epoch 2/8
281/281 [==============================] - 277s 986ms/step - loss: 2.0747 - accuracy: 0.2646 - val_loss: 2.0332 - val_accuracy: 0.2762 - _timestamp: 1648770942.0000 - _runtime: 607.0000
Epoch 3/8
255/281 [==========================>...] - ETA: 22s - loss: 2.0054 - accuracy: 0.2898

In [ ]:
wandb.agent(sweep_id,train,entity="pandp",project = 'cs6910-As-2', count=40)

wandb: Agent Starting Run: 37dfo6sg with config:
wandb: 	batch_norm: False
wandb: 	dense_neuron: 512
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[5, 5], [5, 5], [5, 5], [5, 5], [5, 5]]
wandb: 	filters: [32, 32, 32, 32, 32]
wandb: 	lr: 1e-05
wandb: Currently logged in as: pranab045 (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Epoch 1/8
281/281 [==============================] - 2269s 8s/step - loss: 2.2997 - accuracy: 0.1254 - val_loss: 2.2926 - val_accuracy: 0.1452 - _timestamp: 1648884921.0000 - _runtime: 2289.0000
Epoch 2/8
281/281 [==============================] - 296s 1s/step - loss: 2.2732 - accuracy: 0.1618 - val_loss: 2.2478 - val_accuracy: 0.1855 - _timestamp: 1648885251.0000 - _runtime: 2619.0000
Epoch 3/8
281/281 [==============================] - 290s 1s/step - loss: 2.2274 - accuracy: 0.1822 - val_loss: 2.2087 - val_accuracy: 0.2137 - _timestamp: 1648885566.0000 - _runtime: 2934.0000
Epoch 4/8
281/281 [==============================] - 290s 1s/step - loss: 2.1908 - accuracy: 0.2040 - val_loss: 2.1635 - val_accuracy: 0.2268 - _timestamp: 1648885888.0000 - _runtime: 3256.0000
Epoch 5/8
281/281 [==============================] - 290s 1s/step - loss: 2.1525 - accuracy: 0.2278 - val_loss: 2.1318 - val_accuracy: 0.2319 - _timestamp: 1648886178.0000 - _runtime: 3546.0000
Epoch 6/8
281/281 [==========

accuracy,▁▃▄▅▆▇▇█
epoch,▁▂▃▄▅▆▇█
loss,█▇▆▅▃▃▂▁
val_accuracy,▁▃▅▆▆▇▆█
val_loss,█▇▆▄▃▂▂▁
accuracy,0.26352
best_epoch,7
best_val_accuracy,0.27218
epoch,7
loss,2.07345
val_accuracy,0.27218


wandb: Agent Starting Run: 0ipip6s9 with config:
wandb: 	batch_norm: False
wandb: 	dense_neuron: 512
wandb: 	dropout: 0.2
wandb: 	epochs: 8
wandb: 	filter_size: [[7, 7], [7, 7], [7, 7], [7, 7], [7, 7]]
wandb: 	filters: [64, 32, 16, 8, 4]
wandb: 	lr: 1e-05


...

wandb: Agent Starting Run: zuqi9s01 with config:
wandb: 	batch_norm: False
wandb: 	dense_neuron: 256
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	filters: [64, 64, 64, 64, 64]
wandb: 	lr: 1e-05


Epoch 1/8
281/281 [==============================] - 295s 1s/step - loss: 2.2997 - accuracy: 0.1212 - val_loss: 2.2924 - val_accuracy: 0.1633 - _timestamp: 1648889865.0000 - _runtime: 304.0000
Epoch 2/8
281/281 [==============================] - 294s 1s/step - loss: 2.2810 - accuracy: 0.1637 - val_loss: 2.2597 - val_accuracy: 0.1845 - _timestamp: 1648890187.0000 - _runtime: 626.0000
Epoch 3/8
281/281 [==============================] - 294s 1s/step - loss: 2.2364 - accuracy: 0.1838 - val_loss: 2.2090 - val_accuracy: 0.1935 - _timestamp: 1648890481.0000 - _runtime: 920.0000
Epoch 4/8
281/281 [==============================] - 293s 1s/step - loss: 2.1955 - accuracy: 0.2023 - val_loss: 2.1694 - val_accuracy: 0.2268 - _timestamp: 1648890802.0000 - _runtime: 1241.0000
Epoch 5/8
281/281 [==============================] - 294s 1s/step - loss: 2.1685 - accuracy: 0.2181 - val_loss: 2.1514 - val_accuracy: 0.2137 - _timestamp: 1648891125.0000 - _runtime: 1564.0000
Epoch 6/8
281/281 [==============

accuracy,▁▃▄▅▆▇▇█
epoch,▁▂▃▄▅▆▇█
loss,█▇▆▄▃▂▂▁
val_accuracy,▁▃▃▆▅▆▆█
val_loss,█▇▅▃▃▂▁▁
accuracy,0.2475
best_epoch,7
best_val_accuracy,0.25202
epoch,7
loss,2.11559
val_accuracy,0.25202


wandb: Agent Starting Run: i042uv05 with config:
wandb: 	batch_norm: False
wandb: 	dense_neuron: 512
wandb: 	dropout: 0.15
wandb: 	epochs: 8
wandb: 	filter_size: [[5, 5], [5, 5], [5, 5], [5, 5], [5, 5]]
wandb: 	filters: [32, 32, 32, 32, 32]
wandb: 	lr: 0.0001


Epoch 1/8
281/281 [==============================] - 290s 1s/step - loss: 2.2271 - accuracy: 0.1664 - val_loss: 2.1489 - val_accuracy: 0.2198 - _timestamp: 1648892428.0000 - _runtime: 298.0000
Epoch 2/8
281/281 [==============================] - 288s 1s/step - loss: 2.1018 - accuracy: 0.2354 - val_loss: 2.0846 - val_accuracy: 0.2379 - _timestamp: 1648892716.0000 - _runtime: 586.0000
Epoch 3/8
281/281 [==============================] - 288s 1s/step - loss: 2.0497 - accuracy: 0.2632 - val_loss: 2.0238 - val_accuracy: 0.2742 - _timestamp: 1648893004.0000 - _runtime: 874.0000
Epoch 4/8
281/281 [==============================] - 287s 1s/step - loss: 1.9955 - accuracy: 0.2930 - val_loss: 1.9881 - val_accuracy: 0.2903 - _timestamp: 1648893325.0000 - _runtime: 1195.0000
